In [ ]:
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-2c3e81ee-7430-b08f-2af6-c27e99dc475d)


In [ ]:
#connect drive with colab notebook/
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [ ]:
!cp /content/drive/MyDrive/covid/train.zip /content/
!cp /content/drive/MyDrive/covid/val.zip /content/

In [ ]:
%%capture 
!unzip /content/train.zip
!rm /content/train.zip
!mv /content/resized /content/train


In [ ]:
%%capture 
!unzip /content/val.zip -d val
!rm /content/val.zip


In [ ]:
import pandas as pd
import numpy as np
from glob import glob
import cv2


In [ ]:
import torch 
from torchvision import transforms 
import torchvision.models as models
import torch.nn as nn

In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Fri May 21 09:21:10 2021

@author: TAC
"""

import torch 
from torchvision import transforms 
import torchvision.models as models
import torch.nn as nn
from glob import glob
import re
from sklearn.utils import resample
import numpy as np


from PIL import Image
class dfloader(torch.utils.data.Dataset):
    
    def __init__(self,path,label,n_samples=100,transform=None):
        self.path = path    
        self.label=label
        self.transform=transform
        self.n_samples=n_samples
        
    def __getitem__(self,index):
        if type(index) == torch.Tensor:
          index = index.item()
        #choose a path and correponding label
        path_ind=self.path[index]
        label_ind=self.label[index]
        #now open that path folder
        img_path=glob(path_ind+'*.jpg')
        #find all image ids#
        #replace '\\' with '\' depend upon os
        img_idx=[int(re.findall(r'\d+', i.split('/')[-1])[0]) for i in img_path]   
        #sort all path accordinh to sequence 
        img_idx, img_path = zip(*sorted(zip(img_idx, img_path)))
        #now resample the img_idx
        if len(img_idx)>self.n_samples:
            img_idx=sorted(resample(img_idx,replace=False,n_samples=self.n_samples))
        else:
            img_idx=sorted(resample(img_idx,replace=True,n_samples=self.n_samples))
        #now find correponding image_path
        img_path=[img_path[i] for i in img_idx]
        
        #now loop all image_path
        #img_list = [Image.open(p).convert('L') for p in img_path]
        img_list=[]
        for p in img_path:
            img=Image.open(p).convert('L')
            if self.transform:
                img_list.append(self.transform(img))
        
        #label_list = [label_ind]*len(img_list)
      
        #convert to array
        image_array=np.stack((img_list))
        image_array=np.moveaxis(image_array,0,3)
        #label_array=np.array(label_list)
        
        return image_array ,label_ind
    
    def __len__(self):
        return len(self.path)

# def my_collate(batch):
#     #move axis
#     data=np.concatenate([i[0] for i in batch])
#     label=np.concatenate([[i[1]]*len(i[0]) for i in batch])
#     return torch.tensor(data,dtype=torch.float),torch.tensor(label)
    



In [ ]:
from PIL import Image
aug=transforms.Compose([
    #transforms.Resize((224,224)), a;ready resized
    #transforms.Grayscale(),#in code gray scales
    # transforms.RandomHorizontalFlip(p=0.5),
    # transforms.RandomVerticalFlip(p=0.5),
    # transforms.RandomRotation(degrees=5),
    
    transforms.ToTensor(),
    transforms.Normalize([0.4, ], [0.3,]),
                        ])


In [ ]:
#first list all folders and thier labels
def prepare_data(path):
  x=glob(path+'/*/')
  _pos_path=[folder for folder in glob(x[0]+'/*/')]
  _neg_path=[folder for folder in glob(x[1]+'/*/')]

  _pos_label=[0]*len(_pos_path)
  _neg_label=[1]*len(_neg_path)

  path=_pos_path+_neg_path
  label=_pos_label+_neg_label
  return path,label

In [ ]:
train_path,train_label=prepare_data('/content/train')
val_path,val_label=prepare_data('/content/val')

In [ ]:
print(len(train_path),len(train_label))
print(len(val_path),len(val_label))

1552 1552
374 374


In [ ]:
import torch
import torch.nn as nn
class Fire1(nn.Module):
   
    def __init__(self, in_channels, squeeze_channels,expand_channels):
        super(Fire1, self).__init__()

        # squeeze 
        self.squeeze = nn.Conv3d(in_channels, squeeze_channels, kernel_size=1)
        self.relu = nn.LeakyReLU()

        # expand
        self.expand_1x1 =nn.Conv3d(squeeze_channels, expand_channels, kernel_size=1)
        self.expand_3x3 =nn.Conv3d(squeeze_channels, expand_channels, kernel_size=3,padding=1)
        

    def forward(self, x):
        x = self.squeeze(x)
        x= self.relu(x)
        x = torch.cat([self.expand_1x1(x),self.expand_3x3(x)], dim=1)
        x = self.relu(x)
        return x
    
    
class MyModelV3(torch.nn.Module):
    
    def __init__(self):
        super(MyModelV3, self).__init__()
        self.net = torch.nn.Sequential(
                 nn.Conv3d(in_channels=1,out_channels=32,kernel_size=3,stride=2),
                 nn.MaxPool3d(kernel_size=3,stride=2),
                 nn.LeakyReLU(),
                 nn.BatchNorm3d(32),
                 Fire1(in_channels=32, squeeze_channels=16,expand_channels=32),
                 Fire1(in_channels=64, squeeze_channels=16,expand_channels=64),
                 nn.MaxPool3d(kernel_size=3,stride=2),
                 #nn.BatchNorm2d(128),
                 Fire1(in_channels=128, squeeze_channels=32,expand_channels=96),
                 Fire1(in_channels=192, squeeze_channels=32,expand_channels=128),
                 nn.MaxPool3d(kernel_size=3,stride=2),
                 #nn.BatchNorm2d(256),
                 Fire1(256, 48, 160),
                 Fire1(320, 48, 160),
                 #nn.BatchNorm2d(320),
                 nn.Conv3d(in_channels=320,out_channels=128*2,kernel_size=1,stride=2),
                 nn.LeakyReLU(),
                 nn.Conv3d(in_channels=128*2,out_channels=32,kernel_size=1,stride=1),
                 nn.LeakyReLU(),
                 nn.Conv3d(in_channels=32,out_channels=16,kernel_size=1,stride=1),
                 nn.LeakyReLU(),
                 #nn.Conv2d(in_channels=32,out_channels=16,kernel_size=1,stride=1),
                 nn.Flatten(),
                 #PrintShape(),
                 nn.Linear(2352, 1024),
                 nn.LeakyReLU(),
                 nn.Linear(1024, 1),
                 # nn.Linear(512*2, 512)
                
               
                )        
    def forward(self, x):
        return self.net(x)    

x=torch.randn(5,1,224,244,100)
net=MyModelV3()
out=net(x)
out.shape

torch.Size([5, 1])

In [ ]:
from torch.utils.data.dataloader import DataLoader
train=dfloader(train_path,train_label,transform=aug)
train_loader = DataLoader(train,shuffle=True,num_workers=0,batch_size=3)

for i,j in train_loader:
    print(i.shape,j.shape)
    break
    


torch.Size([3, 1, 224, 224, 100]) torch.Size([3])


In [ ]:
from sklearn.metrics import classification_report,f1_score
from time import time

def get_accuracy(y_true, y_pred):
    assert y_true.ndim == 1 and y_true.size() == y_pred.size()
    y_pred = y_pred > 0.5
    return (y_true == y_pred).sum().item() / y_true.size(0)

def fit_train(loader):
    loss_sum=0
    acc_sum=0
    scaler = torch.cuda.amp.GradScaler() 
    for batch in loader:
        img,label=batch
        with torch.cuda.amp.autocast(): 
          img,label = img.to(device),label.to(device)
          out=model(img)
          loss=criterion(out.view(-1),label.float())
        opt.zero_grad()
        scaler.scale(loss).backward()
        #loss.backward()
        scaler.step(opt)
        #opt.step()
        scaler.update()
        loss_sum+=loss.item()
        acc_sum += get_accuracy(label,out.view(-1))
    return loss_sum,acc_sum
def fit_val(loader):
    loss_sum=0
    acc_sum=0
    for batch in loader:
        img,label=batch
        img,label = img.to(device),label.to(device)
        out=model(img)
        loss=criterion(out.view(-1),label.float())
        loss_sum+=loss.item()
        acc_sum += get_accuracy(label,out.view(-1))
    return loss_sum,acc_sum

def fit_test(loader):
  test_pred=[]
  for img,_ in loader:
    test_pred.append(model(img.to(device)))
  test_pred=torch.cat(test_pred,dim=0)
  y_pred=test_pred.cpu().numpy()
  return y_pred


def fit(model,train_loader,val_loader,test_loader=None,epoch=10,scheduler_step=None,verbose=None):
  
  train_loss_plt=[]
  val_loss_plt=[]
  train_acc_plt=[]
  val_acc_plt=[]
  for ep in range(epoch):
    start=time()
    #start training loop
    train_loss,train_acc=fit_train(train_loader)
    #start validation loop
    model.eval()
    with torch.no_grad():
        val_loss,val_acc=fit_val(val_loader)
    end=np.round((time()-start)/60,2) #time in minute
    model.train()

    #calculate print and append the results for plotting purpose
    val_avg_loss=np.round(val_loss/len(val_loader),2)#val loss of all batches of one epoch
    train_avg_loss=np.round(train_loss/len(train_loader),2)# train loss of all batches of one epoch
    train_avg_acc=np.round(train_acc/len(train_loader),2)#train acc of all batches of one epoch
    val_avg_acc=np.round(val_acc/len(val_loader),2)#val acc of all batches of one epoch
    if scheduler_step:
      scheduler.step(val_avg_loss)
    if verbose:
      print('Epoch {}, time {} min , train acc  {}, train loss {} , val acc is {}, loss is {}, learning rate is {} '.format
            (ep,end,train_avg_acc,train_avg_loss,val_avg_acc,val_avg_loss,opt.param_groups[0]['lr']))
    train_loss_plt.append(train_avg_loss)  #append loss of training data  
    val_loss_plt.append(val_avg_loss)     #append loss of validation data
    train_acc_plt.append(train_avg_acc)  #append acc of training data  
    val_acc_plt.append(val_avg_acc)     #append acc of validation data
 
 #test phase
  if test_loader:
      model.eval()
      with torch.no_grad():
         y_pred=fit_test(test_loader)
  
  return [train_loss_plt,val_loss_plt,train_acc_plt,val_acc_plt]

In [ ]:
from torch.utils.data.dataloader import DataLoader
train=dfloader(train_path,train_label,transform=aug)
val=dfloader(val_path,val_label,transform=aug)
train_loader = DataLoader(train,shuffle=True,num_workers=0,batch_size=5)
val_loader = DataLoader(val,shuffle=True,num_workers=0,batch_size=5)

In [ ]:
device=torch.device('cuda')
model=MyModelV3().to(device)
criterion = nn.BCEWithLogitsLoss()
opt=torch.optim.AdamW(params=model.parameters(),lr=0.001)

In [ ]:
res=fit(model,train_loader,val_loader,epoch=10,scheduler_step=False,verbose=True)

Epoch 0, time 6.2 min , train acc  0.44, train loss 0.69 , val acc is 0.44, loss is 0.69, learning rate is 0.001 
Epoch 1, time 6.22 min , train acc  0.44, train loss 0.69 , val acc is 0.44, loss is 0.69, learning rate is 0.001 
Epoch 2, time 6.25 min , train acc  0.44, train loss 0.69 , val acc is 0.44, loss is 0.69, learning rate is 0.001 
Epoch 3, time 6.23 min , train acc  0.44, train loss 0.69 , val acc is 0.44, loss is 0.69, learning rate is 0.001 
Epoch 4, time 6.18 min , train acc  0.44, train loss 0.69 , val acc is 0.44, loss is 0.69, learning rate is 0.001 
Epoch 5, time 6.19 min , train acc  0.44, train loss 0.69 , val acc is 0.44, loss is 0.69, learning rate is 0.001 
Epoch 6, time 6.16 min , train acc  0.44, train loss 0.69 , val acc is 0.44, loss is 0.69, learning rate is 0.001 
Epoch 7, time 6.15 min , train acc  0.44, train loss 0.69 , val acc is 0.44, loss is 0.69, learning rate is 0.001 
Epoch 8, time 6.14 min , train acc  0.44, train loss 0.69 , val acc is 0.44, loss

KeyboardInterrupt: ignored